# Self Study 8: The Kaldor Model

Let's explore the Kaldor model we discussed last week in the lecture. This time, we will use different functional forms for Savings and Investment than the ones you saw in class.

The objective of using these functions is to learn how to explore a nonlinear model using the tools we have already encountered, in order to make sense of a model where analytical solutions may not exist.

Note: The the takeaway from this exercise is not to memorize specific functions, but to understand the methodology for exploring the model:
1. Plot the functions to understand their behavior 
2. Find fixed points visually
3. Look at the Phase Diagram to understand the dynamics of the system
4. What if we assume a simpler functional form? How does the model change? Do the conclusions change?


## The Model

Our economy is described by two variables:

1. **Output ($ Y $)**: The total value of all goods and services produced.
2. **Capital Stock ($ K $)**: The total capital.

Their dyanamics is governed by the following ODEs:

$$
\begin{align*}
\frac{dY}{dt} &= \alpha \left( I(Y, K) - S(Y, K) \right)\\
\frac{dK}{dt} &= I(Y, K) - \delta K
\end{align*}
$$

The way to think about the difference is that production produces two types of output: capital goods and consumption goods. The negative sign in $S$ comes from the fact that savings is the complement of consumption.

where:
- $ \alpha $ is the speed at which Y changes in response to discrepancies in investment and savings.
- $ \delta $ is capital depreciation rate.



### Investment
For our exercise, the investment function $ I(Y, K) $ is given by a cubic function of $Y$ and a linear function of $K$:

$$
I(Y, K) = 3 + \frac{1}{1+\exp(-2.5\cdot(Y-4))} - \frac{K}{2}
$$

Note key features of this function:
1. $I$ is non-linear in $ Y $, but linear in $ K $
2. $I$ increases in $ Y $, up to a maximum
3. $I$ decreases in $ K $


> [...] when there is a **great deal of surplus of capacity**, an increase in activity will not induce entrepreneurs to undertake additional construction: the **rise in profits will not stimulate investment**. ... But it will also be small for unusually high levels of activity, because rising costs of construction, increasing costs and increasing difficulty of borrowing will dissuade entrepreneurs from expanding still faster - at a time when they already have large commitments (Kaldor, 1940, p.180)

### Savings

The savings function $ S(Y, K) $ must be positive and increasing in $ Y $. For our exercise we assume that it only depends on $Y$ not on $K$ and will use the following function:

$$
S(Y, K) = sY
$$

where $ s $ is the marginal propensity to save, indicating the fraction of output that is saved.


> [...] when incomes are unusually low, savings are cut drastically, and below a certain level of income they will be negative. When incomes are unusually high, people are likely to save not only a higher amount, but also a larger proportion of their income (Kaldor, 1940, p.141)

## Section 1: Basic Dynamics

### Task 1: Define functions

- $S$
- $I$
- $\frac{dY}{dt}$
- $\frac{dK}{dt}$

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact
from scipy.integrate import odeint
from ipywidgets import fixed

In [2]:
def investment(Y, K, linear=False):
    if linear:
        return 3 + Y/10 - K/5
    else:
        return 1/(1+np.exp(-2.5*(Y-4))) + 3 - K/2


def savings(Y, s):
    return s * Y


def dYdt(I, S, alpha):
    return alpha * (I - S)

def dKdt(K, I, delta):
    return I - delta * K

### Task 2: S/I curves

- Plot the savings and investment curves for $Y$
- Explore parameters for $a$, $b$, $c$, $s$ and $\delta$ to see how they affect the number of equilibria. 

In [3]:
@interact(s=(0.1, 1, 0.1), kk=(0.1, 6, 0.1))
def plot_si(s, kk=1.):
    Y = np.linspace(0, 10, 100)
    # make two subplots one for varying K and one for varying Y
    fig, ax = plt.subplots(1, 1, figsize=(4, 4))
    ax.plot(Y, investment(Y, K=kk), label='Investment')
    ax.plot(Y, savings(Y, s=s), label='Savings')
    ax.set_xlim(0)
    ax.set_ylim(0)
    plt.legend()
    plt.xlabel('Output (Y)')
    plt.ylabel('Investment (I) / Savings (S)')

interactive(children=(FloatSlider(value=0.5, description='s', max=1.0, min=0.1), FloatSlider(value=1.0, descri…

- **Questions:** When chaning $K$ and $s$ what do you observe? What are the implications for $Y$?
- **Answer:** 

### Task 3: Dynamnics of Y

Specifically, plot:
- $\dot{Y}$ vs $Y$ for different values of $K$, $s$.
- $\dot{K}$ vs $K$ for different values of $Y$, $s$ and $\delta$.

This plot will help you understand the dynamics of the system, without having to solve the ODEs.


Note: $\dot{Y}$ is just an other way of writing $\frac{dY}{dt}$.


In [4]:
import matplotlib.pyplot as plt
from ipywidgets import interact

@interact(s=(0.1, 1, 0.01), kk=(0.1, 6, 0.1), alpha=(0.01, 4.0, 0.01), delta=(0.01, 1, .01))
def plot_change_y(s, kk, alpha, delta):
    Y = np.linspace(0, 10, 100)
    # make two subplots one for varying K and one for varying Y
    fig, ax = plt.subplots(1, 1, figsize=(4, 4))
    I = investment(Y, K=kk)
    S = savings(Y, s)
    ax.plot(Y, dYdt(I, S, alpha), label="$\dot{Y}$")
    ax.plot(Y, dKdt(kk, I, delta), label="$\dot{K}$")
    plt.xlabel('$Y$')
    plt.ylabel('$\dot{Y}$, $\dot{K}$')
    plt.legend()
    
    # add horizontal line at 0
    plt.axhline(0, color='k')

interactive(children=(FloatSlider(value=0.55, description='s', max=1.0, min=0.1, step=0.01), FloatSlider(value…

- **Question:** What are the points at which $\dot{Y} = 0$ and $\dot{K} = 0$? What is the dynamics of $Y$ around these points?
- **Answer:** If the derivative is above 0, then the variable is increasing. If the derivative is below 0, then the variable is decreasing. Therefore we can see that the dynamics of $Y$ around the fixed points is never stable for K, and (may with the correct parameters) have 3 fixed points for Y, stability depends on $s$.

- **Question:** What happens to the fixed points when $\alpha$ changes? What does it mean for the dynamics of the system?
- **Answer:**

## Section 2: Phase Diagram

Plot the phase diagram for the system.

In [5]:
def kaldor_system(YK, t, s, alpha, delta):
    Y, K = YK
    I = investment(Y, K)
    S = savings(Y, s)
    return dYdt(I, S, alpha=alpha), dKdt(K, I, delta=delta)

In [6]:
def solve_and_plot(alpha, delta, s):
    t = np.linspace(0, 200, 400)
    u0 = [1, 1]
    sol = odeint(kaldor_system, u0, t, args=(s, alpha, delta))
    fig, ax = plt.subplots(1, 2, figsize=(8, 4))
    ax[0].plot(t, sol[:, 0], label='Output')
    ax[0].plot(t, sol[:, 1], label='Capital')
    ax[0].set_xlabel('Time')
    ax[0].set_ylabel('Output / Capital')
    
    ax[1].plot(sol[:, 0], sol[:, 1], label='Output', color='red')
    ax[1].set_xlabel('Output')
    ax[1].set_ylabel('Capital')
    ax[0].legend()
    
    # plot origin
    ax[1].plot(*u0, 'ko')
    
    # streamplot
    Y, K = np.meshgrid(np.linspace(0, 6, 30), np.linspace(0, 6, 30))
    dYdt, dKdt = kaldor_system((Y, K), t=None, s=s, alpha=alpha, delta=delta)
    ax[1].streamplot(Y, K, dYdt, dKdt, color='gray', density=1.1)
    ax[1].set_xlabel('Output')
    ax[1].set_ylabel('Capital')
    

In [7]:
# Use these parameters to start, watch at what happens at delta=0.2
interact(solve_and_plot, alpha=fixed(2.2), delta=(0, 1.0, 0.01), s=fixed(0.25))


interactive(children=(FloatSlider(value=0.5, description='delta', max=1.0, step=0.01), Output()), _dom_classes…

<function __main__.solve_and_plot(alpha, delta, s)>

## Section 3: What if $I$ is linear also in $Y$?

Rerun the above code but with the following investment function:
$$
I(Y, K) = 3 + \frac{Y}{3} - \frac{K}{2}
$$

- **Question:** What are the point(s) at which $\dot{Y} = 0$ and $\dot{K} = 0$? What is the dynamics of $Y$ around these points?
- **Answer:**

- **Question:** If we find that $Y$ is unstable can be locked in a cycle, what does it mean for the economy?
- **Answer:**

## Discussion of changing $\delta$ vs $\alpha$ what is the mean output?

In [8]:
## An alternative to the continuous functions we have used is a piecewise function

def saving_piecewise(Y, K, omega=.1, c00=1, c1=.1, c2=.5, c3=.1, k1=3, k2=5):
    # compute the breakpoints to have a smooth transition
    c01 = c00 + c1 * k1
    c02 = c01 + c2 * (k2 - k1)

    # Calculate values for each segment
    segment1 = c00 + c1 * Y - omega * K
    segment2 = c01 + c2 * (Y - k1) - omega * K
    segment3 = c02 + c3 * (Y - k2) - omega * K

    # apply the correct segment calculation
    result = np.where(Y < k1, segment1, np.where(Y < k2, segment2, segment3))
    return result


@interact(omega=(0.01, 0.5, 0.01), c00=(0, 10, .1), c1=(0.01, 0.5, 0.01), c2=(0.01, 0.5, 0.01), c3=(0.01, 0.5, 0.01), kk=(0.1, 6, 0.1))
def plot_piecewise(kk, omega, c00, c1, c2, c3):
    Y = np.linspace(0, 10, 100)
    S = saving_piecewise(Y, kk, omega, c00, c1, c2, c3)
    plt.plot(Y, S)
    

interactive(children=(FloatSlider(value=3.0000000000000004, description='kk', max=6.0, min=0.1), FloatSlider(v…